In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.sentiment import SentimentIntensityAnalyzer
import re
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    

In [2]:
csv_file_path = 'df_tweets.csv'
df = pd.read_csv(csv_file_path)
df.shape

(164168, 12)

In [3]:
df

,full_text,created_at,hashtags,user_location,friends_count,followers_count,reply_count,retweet_count,favorite_count,is_with_url,HYDROXYCHLOROQUINE,query_string
0,From the January 2020 Arthritis Care &amp; Res...,2020-01-03,NaN,NaN,5,4413,0,2,10,1,1,HYDROXYCHLOROQUINE lang:en until:2020-01-03 si...
1,@TheLancetRheum Classic “Bulls eye retinopathy...,2020-01-07,NaN,NaN,20,10,0,0,1,0,1,HYDROXYCHLOROQUINE lang:en until:2020-01-08 si...
2,Dr. Anil Pareek -Research has revealed that hy...,2020-01-12,NaN,"Hyderabad, India",25050,32355,0,0,0,1,1,HYDROXYCHLOROQUINE lang:en until:2020-01-13 si...
3,A randomised phase II trial of hydroxychloroqu...,2020-01-13,NaN,NaN,33,16,0,0,0,1,1,HYDROXYCHLOROQUINE lang:en until:2020-01-14 si...
4,@GeekyJules What are the side effects of the b...,2020-01-14,NaN,"Colorado, USA",2619,1075,2,0,0,0,1,HYDROXYCHLOROQUINE lang:en until:2020-01-14 si...
...,...,...,...,...,...,...,...,...,...,...,...,...
164163,@ritatay29333372 @edhunter54 @specialistgi @je...,2020-12-30,NaN,NaN,102,12,1,0,1,0,1,HYDROXYCHLOROQUINE lang:en until:2020-12-30 si...
164164,@AlienXperience @SarahKSilverman @marcorubio I...,2020-12-30,NaN,NaN,1645,12921,1,0,1,0,1,HYDROXYCHLOROQUINE lang:en until:2020-12-30 si...
164165,@hcphtx @SOnyiego @abc13houston @FOX26Houston ...,2020-12-30,NaN,CA,262,156,0,0,0,1,1,HYDROXYCHLOROQUINE lang:en until:2020-12-30 si...
164166,@DrTomFrieden THE COVID-19 RESPONSE IS MEDICAL...,2020-12-30,NaN,CA,262,156,0,0,1,1,1,HYDROXYCHLOROQUINE lang:en until:2020-12-30 si...


In [4]:
numerical_columns = df.select_dtypes(include=['number'])
numerical_columns

,friends_count,followers_count,reply_count,retweet_count,favorite_count,is_with_url,HYDROXYCHLOROQUINE
0,5,4413,0,2,10,1,1
1,20,10,0,0,1,0,1
2,25050,32355,0,0,0,1,1
3,33,16,0,0,0,1,1
4,2619,1075,2,0,0,0,1
...,...,...,...,...,...,...,...
164163,102,12,1,0,1,0,1
164164,1645,12921,1,0,1,0,1
164165,262,156,0,0,0,1,1
164166,262,156,0,0,1,1,1


In [5]:
text_columns = df.select_dtypes(include=['object'])
text_columns

,full_text,created_at,hashtags,user_location,query_string
0,From the January 2020 Arthritis Care &amp; Res...,2020-01-03,NaN,NaN,HYDROXYCHLOROQUINE lang:en until:2020-01-03 si...
1,@TheLancetRheum Classic “Bulls eye retinopathy...,2020-01-07,NaN,NaN,HYDROXYCHLOROQUINE lang:en until:2020-01-08 si...
2,Dr. Anil Pareek -Research has revealed that hy...,2020-01-12,NaN,"Hyderabad, India",HYDROXYCHLOROQUINE lang:en until:2020-01-13 si...
3,A randomised phase II trial of hydroxychloroqu...,2020-01-13,NaN,NaN,HYDROXYCHLOROQUINE lang:en until:2020-01-14 si...
4,@GeekyJules What are the side effects of the b...,2020-01-14,NaN,"Colorado, USA",HYDROXYCHLOROQUINE lang:en until:2020-01-14 si...
...,...,...,...,...,...
164163,@ritatay29333372 @edhunter54 @specialistgi @je...,2020-12-30,NaN,NaN,HYDROXYCHLOROQUINE lang:en until:2020-12-30 si...
164164,@AlienXperience @SarahKSilverman @marcorubio I...,2020-12-30,NaN,NaN,HYDROXYCHLOROQUINE lang:en until:2020-12-30 si...
164165,@hcphtx @SOnyiego @abc13houston @FOX26Houston ...,2020-12-30,NaN,CA,HYDROXYCHLOROQUINE lang:en until:2020-12-30 si...
164166,@DrTomFrieden THE COVID-19 RESPONSE IS MEDICAL...,2020-12-30,NaN,CA,HYDROXYCHLOROQUINE lang:en until:2020-12-30 si...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164168 entries, 0 to 164167
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   full_text           164168 non-null  object
 1   created_at          164168 non-null  object
 2   hashtags            31482 non-null   object
 3   user_location       109595 non-null  object
 4   friends_count       164168 non-null  int64 
 5   followers_count     164168 non-null  int64 
 6   reply_count         164168 non-null  int64 
 7   retweet_count       164168 non-null  int64 
 8   favorite_count      164168 non-null  int64 
 9   is_with_url         164168 non-null  int64 
 10  HYDROXYCHLOROQUINE  164168 non-null  int64 
 11  query_string        164168 non-null  object
dtypes: int64(7), object(5)
memory usage: 15.0+ MB


In [7]:
df.describe()

,friends_count,followers_count,reply_count,retweet_count,favorite_count,is_with_url,HYDROXYCHLOROQUINE
count,1.641680e+05,1.641680e+05,164168.000000,164168.000000,164168.000000,164168.000000,164168.0
mean,2.202445e+03,2.755409e+04,6.870072,25.337496,74.263462,0.232092,1.0
std,1.131294e+04,9.139877e+05,268.086344,669.759656,2050.033366,0.422168,0.0
min,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,1.0
25%,1.930000e+02,7.800000e+01,0.000000,0.000000,0.000000,0.000000,1.0
50%,5.880000e+02,3.740000e+02,0.000000,0.000000,0.000000,0.000000,1.0
75%,1.863000e+03,1.628000e+03,1.000000,0.000000,2.000000,0.000000,1.0
max,1.607282e+06,8.572541e+07,69341.000000,101604.000000,374415.000000,1.000000,1.0


### Deleting uncessary columns 

In [8]:
df

,full_text,created_at,hashtags,user_location,friends_count,followers_count,reply_count,retweet_count,favorite_count,is_with_url,HYDROXYCHLOROQUINE,query_string
0,From the January 2020 Arthritis Care &amp; Res...,2020-01-03,NaN,NaN,5,4413,0,2,10,1,1,HYDROXYCHLOROQUINE lang:en until:2020-01-03 si...
1,@TheLancetRheum Classic “Bulls eye retinopathy...,2020-01-07,NaN,NaN,20,10,0,0,1,0,1,HYDROXYCHLOROQUINE lang:en until:2020-01-08 si...
2,Dr. Anil Pareek -Research has revealed that hy...,2020-01-12,NaN,"Hyderabad, India",25050,32355,0,0,0,1,1,HYDROXYCHLOROQUINE lang:en until:2020-01-13 si...
3,A randomised phase II trial of hydroxychloroqu...,2020-01-13,NaN,NaN,33,16,0,0,0,1,1,HYDROXYCHLOROQUINE lang:en until:2020-01-14 si...
4,@GeekyJules What are the side effects of the b...,2020-01-14,NaN,"Colorado, USA",2619,1075,2,0,0,0,1,HYDROXYCHLOROQUINE lang:en until:2020-01-14 si...
...,...,...,...,...,...,...,...,...,...,...,...,...
164163,@ritatay29333372 @edhunter54 @specialistgi @je...,2020-12-30,NaN,NaN,102,12,1,0,1,0,1,HYDROXYCHLOROQUINE lang:en until:2020-12-30 si...
164164,@AlienXperience @SarahKSilverman @marcorubio I...,2020-12-30,NaN,NaN,1645,12921,1,0,1,0,1,HYDROXYCHLOROQUINE lang:en until:2020-12-30 si...
164165,@hcphtx @SOnyiego @abc13houston @FOX26Houston ...,2020-12-30,NaN,CA,262,156,0,0,0,1,1,HYDROXYCHLOROQUINE lang:en until:2020-12-30 si...
164166,@DrTomFrieden THE COVID-19 RESPONSE IS MEDICAL...,2020-12-30,NaN,CA,262,156,0,0,1,1,1,HYDROXYCHLOROQUINE lang:en until:2020-12-30 si...


In [9]:
df.drop(columns=["HYDROXYCHLOROQUINE", "query_string"], inplace=True)
df

,full_text,created_at,hashtags,user_location,friends_count,followers_count,reply_count,retweet_count,favorite_count,is_with_url
0,From the January 2020 Arthritis Care &amp; Res...,2020-01-03,NaN,NaN,5,4413,0,2,10,1
1,@TheLancetRheum Classic “Bulls eye retinopathy...,2020-01-07,NaN,NaN,20,10,0,0,1,0
2,Dr. Anil Pareek -Research has revealed that hy...,2020-01-12,NaN,"Hyderabad, India",25050,32355,0,0,0,1
3,A randomised phase II trial of hydroxychloroqu...,2020-01-13,NaN,NaN,33,16,0,0,0,1
4,@GeekyJules What are the side effects of the b...,2020-01-14,NaN,"Colorado, USA",2619,1075,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...
164163,@ritatay29333372 @edhunter54 @specialistgi @je...,2020-12-30,NaN,NaN,102,12,1,0,1,0
164164,@AlienXperience @SarahKSilverman @marcorubio I...,2020-12-30,NaN,NaN,1645,12921,1,0,1,0
164165,@hcphtx @SOnyiego @abc13houston @FOX26Houston ...,2020-12-30,NaN,CA,262,156,0,0,0,1
164166,@DrTomFrieden THE COVID-19 RESPONSE IS MEDICAL...,2020-12-30,NaN,CA,262,156,0,0,1,1


In [10]:
df.shape

(164168, 10)

### Setting thereshold remove columns with missing values 

In [11]:
# removing all the columns with more than 80% missing values
def deletecols(df):
    thereshold = 0.8 * len(df)
    columns_to_drop = df.columns[df.isnull().sum() > thereshold]
    df.drop(columns=columns_to_drop, inplace=True ) 
    
deletecols(df)
df.shape

(164168, 9)

### Removing duplicates 

In [12]:
df = df.drop_duplicates(subset=["full_text", "created_at", "reply_count", "favorite_count"])
df.reset_index(drop=True, inplace=True)
original_shape = (164168, 9)
new_shape = df.shape
duplicates_deleted = original_shape[0] - new_shape[0]
print("Total duplicates deleted:", duplicates_deleted)
df

Total duplicates deleted: 1277


,full_text,created_at,user_location,friends_count,followers_count,reply_count,retweet_count,favorite_count,is_with_url
0,From the January 2020 Arthritis Care &amp; Res...,2020-01-03,NaN,5,4413,0,2,10,1
1,@TheLancetRheum Classic “Bulls eye retinopathy...,2020-01-07,NaN,20,10,0,0,1,0
2,Dr. Anil Pareek -Research has revealed that hy...,2020-01-12,"Hyderabad, India",25050,32355,0,0,0,1
3,A randomised phase II trial of hydroxychloroqu...,2020-01-13,NaN,33,16,0,0,0,1
4,@GeekyJules What are the side effects of the b...,2020-01-14,"Colorado, USA",2619,1075,2,0,0,0
...,...,...,...,...,...,...,...,...,...
162886,@ritatay29333372 @edhunter54 @specialistgi @je...,2020-12-30,NaN,102,12,1,0,1,0
162887,@AlienXperience @SarahKSilverman @marcorubio I...,2020-12-30,NaN,1645,12921,1,0,1,0
162888,@hcphtx @SOnyiego @abc13houston @FOX26Houston ...,2020-12-30,CA,262,156,0,0,0,1
162889,@DrTomFrieden THE COVID-19 RESPONSE IS MEDICAL...,2020-12-30,CA,262,156,0,0,1,1


### Cleaning Text

In [13]:
nltk.download('vader_lexicon')

#functions for pre processing to clean the text 

def clean_text(sentence):
    sentence = sentence.lower()
    sentence = sentence.strip()
    return sentence

#---clean up html elements and entities: e.g. <html> </html> &nbsp;
def cleanHtml(sentence):
    #cleanr = re.compile('<.*?>')
    cleanr = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext


#---function to clean the word of any punctuation or special characters
def cleanPunc(sentence): 
    cleaned = re.sub(r'[?|!|\'|"|#]',r' ',sentence)
    cleaned = re.sub(r'[.|,|:|;|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned


def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent

#---stop words removing 
stop_words = set(stopwords.words('english'))
stop_words.update(['  ', 'zero','one','two','three','four','five','six','seven','eight','nine','ten','may','also','across','among','beside','however','yet','within'])
re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)

def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)


#--- sentence stemering
stemmer = SnowballStemmer("english")
def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence


def clean_text_columns(full_text):
    full_text = full_text.apply(cleanHtml)
    full_text = full_text.apply(cleanPunc)
    full_text = full_text.apply(keepAlpha)
    full_text = full_text.apply(removeStopWords)
    full_text = full_text.apply(stemming)
    return full_text

df['full_text'] = clean_text_columns(df['full_text'])
df

    

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/jibrankhan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,full_text,created_at,user_location,friends_count,followers_count,reply_count,retweet_count,favorite_count,is_with_url
0,januari arthriti care research clinic structur...,2020-01-03,NaN,5,4413,0,2,10,1
1,thelancetrheum classic bull eye retinopathi ca...,2020-01-07,NaN,20,10,0,0,1,0
2,dr anil pareek research reveal hydroxychloroqu...,2020-01-12,"Hyderabad, India",25050,32355,0,0,0,1
3,randomis phase ii trial hydroxychloroquin imat...,2020-01-13,NaN,33,16,0,0,0,1
4,geekyjul side effect biolog still methotrex su...,2020-01-14,"Colorado, USA",2619,1075,2,0,0,0
...,...,...,...,...,...,...,...,...,...
162886,ritatay edhunt specialistgi jennybethm labeach...,2020-12-30,NaN,102,12,1,0,1,0
162887,alienxperi sarahksilverman marcorubio put stoc...,2020-12-30,NaN,1645,12921,1,0,1,0
162888,hcphtx sonyiego abc houston fox houston khou k...,2020-12-30,CA,262,156,0,0,0,1
162889,drtomfrieden covid respons medic polit fraud e...,2020-12-30,CA,262,156,0,0,1,1


### Adding new column State and populating it with supportive file

In [14]:
%%time

state_lookup = pd.read_csv("state_full.csv")

# Function to extract state abbreviations from user locations
def extract_state(location):
    if pd.isna(location):
        return None
    
    location = location.lower()
    
    # Iterating through the lookup table
    for _, row in state_lookup.iterrows():
        if row['shortState'].lower() in location or row['city'].lower() in location:
            return row['shortState']
    
    return None

# Applying  the function to populate the new state column
df['state'] = df['user_location'].apply(extract_state)

df


KeyboardInterrupt: 

In [ ]:
# Save the DataFrame with the new column
df.to_csv("updated_tweets.csv", index=False)